In [6]:
import itertools


In [7]:
items = [1, 2, 3]
for item in itertools.permutations(items, 3):
    print(item)


(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [8]:
for item in itertools.combinations(items, 2):
    print(item)


(1, 2)
(1, 3)
(2, 3)


In [9]:
for item in itertools.combinations_with_replacement(items, 2):
    print(item)


(1, 1)
(1, 2)
(1, 3)
(2, 2)
(2, 3)
(3, 3)


In [15]:
ab = ['a', 'b']
cd = ['c', 'd']
# Produce the product of two arrays
for item in itertools.product(ab, cd):
    print(item)


('a', 'c')
('a', 'd')
('b', 'c')
('b', 'd')


In [58]:

# 1. Defining a function that takes params and return result+params

import import_ipynb
from strategy_base import TradeStrategyBase

def execute(strategy: TradeStrategyBase,
         keep_stock_threshold: int, 
         buy_change_threshold: float):
    """
    :param keep_stock_threshold: 
    :param buy_change_threshold: 
    :return:  
    """
    
    # Use class method to set keep_stock_threshold
    strategy.set_keep_stock_threshold(keep_stock_threshold)
    
    # Use static method to set buy_change_threshold
    strategy.set_buy_change_threshold(buy_change_threshold)
    
    # Create Trade Days (MasterCard)
    from abupy import ABuSymbolPd, EMarketDataSplitMode
    # List 2 years of MasterCard closing price to list()
    price_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).close.tolist()
    date_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).date.tolist()
    price_array[-5:], date_array[-5:]
    
    from tradedays import StockTradeDays
    trade_days = StockTradeDays(price_array, "", date_array)
    # print('trade_days has {} days'.format(len(trade_days)))
    # print('Last trading day: {}'.format(trade_days[-1]))

    # Backtest
    from trade_loopback import TradeLoopBack
    trade_loop_back = TradeLoopBack(trade_days, strategy)
    trade_loop_back.execute_trade()
    
    # Calculate profit after back test
    from functools import reduce
    profit = 0.0 if len(trade_loop_back.profit_array) == 0 else \
        reduce(lambda a, b: a + b, trade_loop_back.profit_array)
    
    # Return profit and the two input params
    from typing import NamedTuple
    result_tuple = NamedTuple("StrategyResult", 
                             [("ROI", float), 
                              ("KeepThreshold", int), 
                              ("ChangeThreshold", float)])
    return result_tuple(round(profit, 6), keep_stock_threshold, round(buy_change_threshold, 6))


In [59]:

# Initialize an instance of Strategy2
import import_ipynb
from strategy_2 import TradeStrategy2
trade_strategy2 = TradeStrategy2()

execute(trade_strategy2, 5, -0.02)


StrategyResult(ROI=0.137, KeepThreshold=5, ChangeThreshold=-0.02)

In [63]:

# range: buy and then hold 2~30 days, step is 2
keep_stock_days_list = range(2, 30, 2)
print('keep_stock Array: {}'.format(list(keep_stock_days_list)))
# range: 'buy_change' from -0.05 to -0.15, aka down 5% to 15%
buy_change_list = [buy_change / 100.0 for buy_change in range(-5, -16, -1)]
print('buy_change Array: {}'.format(buy_change_list))


keep_stock Array: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
buy_change Array: [-0.05, -0.06, -0.07, -0.08, -0.09, -0.1, -0.11, -0.12, -0.13, -0.14, -0.15]


In [66]:
result = []
for keep_stock_threshold, buy_change_threshold in itertools.product(
    keep_stock_days_list, buy_change_list):
    result.append(
        execute(trade_strategy2, keep_stock_threshold, buy_change_threshold)
    )

print('Cartesian product produced {} results'.format(len(result)))


Cartesian product produced 154 results


In [65]:
sorted(result)[::-1][:10]

[StrategyResult(ROI=0.379, KeepThreshold=8, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.306, KeepThreshold=6, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.298, KeepThreshold=20, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.298, KeepThreshold=8, ChangeThreshold=-0.06),
 StrategyResult(ROI=0.293, KeepThreshold=12, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.289, KeepThreshold=10, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.275, KeepThreshold=28, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.272, KeepThreshold=26, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.263, KeepThreshold=16, ChangeThreshold=-0.05),
 StrategyResult(ROI=0.256, KeepThreshold=18, ChangeThreshold=-0.05)]